In [10]:
"""
!pip3 install numpy
!pip3 install pandas
!pip3 install graphviz
!pip3 install pydot
!pip3 install git+https://github.com/LeeKamentsky/python-javabridge.git#egg=javabridge
!pip3 install ipycytoscape
!pip3 install git+git://github.com/bd2kccd/py-causal
"""

'\n!pip3 install numpy\n!pip3 install pandas\n!pip3 install graphviz\n!pip3 install pydot\n!pip3 install git+https://github.com/LeeKamentsky/python-javabridge.git#egg=javabridge\n!pip3 install ipycytoscape\n!pip3 install git+git://github.com/bd2kccd/py-causal\n'

In [11]:
import os
import pandas as pd
import graphviz
import pydot
import numpy as np
from pycausal.pycausal import pycausal as pc
from IPython.display import SVG 
import ipycytoscape
from ipycytoscape import cytoscape
from ipycytoscape import CytoscapeWidget
import ipywidgets as widgets

In [12]:
incols = pd.read_table('controllable.txt')['attributes'].tolist()


# Names of columns (eventually, names of nodes)
df = pd.read_table('totaldata.csv', sep=',', usecols=lambda x: x in incols)
df = df[df["num_deaths"].notnull()]
#df = pd.read_table('mask.csv', sep=',')
#df = df[df["state"] == "Delaware"]
#df.head()
#df.columns.tolist()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (514) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# Drops the column called "state"
#new_df.head()
# Prints the first five values of data frame, zero-indexed

#df = new_df
#!/usr/local/bin/python

from pycausal.pycausal import pycausal as pc
pc = pc()
pc.start_vm()

from pycausal import search as s
tetrad = s.tetradrunner()

"""
tetrad.run(algoId = 'fges', dfs = df, scoreId = 'bdeu-score', dataType = 'discrete',
           maxDegree = 3, faithfulnessAssumed = True, verbose = False, 
           numberResampling = 5, resamplingEnsemble = 1, addOriginalDataset = True)

tetrad.run(algoId = 'gfci', dfs = df, testId = 'bdeu-test', scoreId = 'bdeu-score',  dataType = 'discrete',
           maxDegree = 3, maxPathLength = -1, completeRuleSetUsed = False, 
           faithfulnessAssumed = True, verbose = False)

           
           tetrad.run(algoId = 'fges', dfs = df, scoreId = 'cg-bic-score', dataType = 'mixed', numCategoriesToDiscretize = 4,
           maxDegree = 3, faithfulnessAssumed = True, verbose = False, 
           numberResampling = 5, resamplingEnsemble = 1, addOriginalDataset = True)


tetrad.run(algoId = 'fges', dfs = df, scoreId = 'sem-bic', dataType = 'continuous',
           maxDegree = -1, faithfulnessAssumed = True, verbose = False)


tetrad.run(algoId = 'fges', dfs = df, scoreId = 'cg-bic-score', dataType = 'discrete',
           maxDegree = 3, faithfulnessAssumed = True, 
           symmetricFirstStep = True, verbose = True)
"""
tetrad.run(algoId = 'fges', dfs = df, scoreId = 'sem-bic', dataType = 'continuous',
           maxDegree = -1, faithfulnessAssumed = True, verbose = False)


graph = tetrad.getTetradGraph()

"""
dot_str = pc.tetradGraphToDot(graph)
graphs = pydot.graph_from_dot_data(dot_str)
graphs[0].write_svg('us-county-health-data.svg')
"""

print(graph)
#print('Graph BIC: {}'.format(graph.getAttribute('BIC')))

Graph Nodes:
num_deaths;chlamydia_rate;cohort_size;high_school_graduation_rate;population;percent_some_college;labor_force;percent_unemployed;income_ratio;percent_severe_housing_problems;severe_housing_cost_burden;overcrowding;percent_drive_alone_to_work;juvenile_arrest_rate;percent_homeowners;percent_severe_housing_cost_burden;candidatevotes;totalvotes;votes_percent

Graph Edges:
1. labor_force --> percent_severe_housing_problems
2. num_deaths --> labor_force
3. num_deaths --> percent_severe_housing_problems
4. percent_homeowners --> labor_force
5. percent_homeowners --> percent_severe_housing_problems
6. percent_homeowners --> percent_some_college
7. percent_some_college --> labor_force
8. percent_unemployed --> percent_severe_housing_problems
9. percent_unemployed --> percent_some_college
10. population --> labor_force
11. population --- num_deaths
12. population --- percent_homeowners
13. population --> percent_some_college
14. totalvotes --> candidatevotes
15. totalvotes --- perce

In [14]:
def getGraphNodeByName(g, n):
    nodes = g.getNodes()
    nit = nodes.iterator()
    while nit.hasNext():
        node = nit.next()
        if node.getName() == n:
            return node
    return None


def buildWidget(g, w):
    w.graph.clear()
    
    edges = g.getEdges()
    eit = edges.iterator()
    connectedNodes = []
    while eit.hasNext():
        edge= eit.next()
        n1 = edge.getNode1().getName()
        n2 = edge.getNode2().getName()
        connectedNodes.append(n1)
        connectedNodes.append(n2)

    nodes = g.getNodes()
    nit = nodes.iterator()
    while nit.hasNext():
        node = nit.next()
        nodeName = node.getName()
        if nodeName in connectedNodes:
            w.graph.add_node(cytoscape.Node(data={"id":nodeName, "label":nodeName}))
    
    eit = edges.iterator()
    while eit.hasNext():
        edge= eit.next()
        n1 = edge.getNode1().getName()
        n2 = edge.getNode2().getName()
        cEdge = cytoscape.Edge(data={"id":f'{n1}_{n2}',"source": n1, "target": n2})
        w.graph.add_edge(cEdge, True, True)
        
    
    #w.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
    w.set_style([
            {
            'selector': 'node',
            'style': {
                'font-family': 'arial',
                'font-size': '10px',
                'label': 'data(label)',
                'background-color': 'blue'
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'width': 4,
                    'line-color': '#9dbaea',
                    'target-arrow-shape': 'triangle',
                    'target-arrow-color': '#9dbaea',
                    'curve-style': 'bezier'
                }
            },
                {
          "selector": "edge.selected",
          "css": {
            "line-color": "red"
          },
        },
        
        {
            "selector": "node.selected",
            "css": {
                "background-color": "red"
            },
        }
    ])

In [15]:
selectedNodes = []
selectedEdges = []

def getCytoGraphEdge(id):
    for ne in widget1.graph.edges:
        if ne.data["id"] == id:
            return ne
    return None

def getCytoGraphNode(id):
    for nd in widget1.graph.nodes:
        if nd.data["id"] == id:
            return nd
    return None

def toggleCytoGraphEdgeClass(id, className, add):
    cEdge = getCytoGraphEdge(id)
    toggleCytoGraphClass(cEdge, className, add)

def toggleCytoGraphNodeClass(id, className, add):
    cNode = getCytoGraphNode(id)
    toggleCytoGraphClass(cNode, className, add)
    
def toggleCytoGraphClass(el, className, add):
    classes = set(el.classes.split(" "))
    if add:
        classes.add(className)
    else:
        if className in classes:
            classes.remove(className)
        
    el.classes = " ".join(classes)

graphbtn = widgets.Button(description="Generate original graph", disabled=False)
deletebtn = widgets.Button(description="Delete Edge", disabled=True)
changebtn = widgets.Button(description="Change Direction", disabled=True)
addbtn = widgets.Button(description="Add Edge", disabled=True)
widget1 = ipycytoscape.CytoscapeWidget()
buildWidget(graph, widget1)

In [16]:
from ipywidgets import Output
from IPython.display import display
from pprint import pformat

out = Output()

def edge_selected(e):
    
    if len(selectedEdges) == 0:
        selectedEdges.append(e)
        with out:
            print(f'selected: {pformat(e["data"])}')
    else:
        toggleCytoGraphEdgeClass(selectedEdges[0]["data"]["id"], "selected", False)
        if selectedEdges[0]["data"]["id"] == e["data"]["id"]:
            selectedEdges.clear()
        else:
            selectedEdges[0] = e
        
        
    empty = len(selectedEdges) == 0
    if not empty:
        toggleCytoGraphEdgeClass(selectedEdges[0]["data"]["id"], "selected", True)
    
    deletebtn.disabled = empty
    changebtn.disabled = empty
    addbtn.disabled = empty
    
def node_selected(n):
    for nd in selectedNodes:
        if nd["data"]["id"] == n["data"]["id"]:
            toggleCytoGraphNodeClass(n["data"]["id"], "selected", False)
            selectedNodes.remove(nd)
            return
    if len(selectedNodes) < 2:
        selectedNodes.append(n)
        with out:
            print(f'selected: {pformat(n["data"])}')
        toggleCytoGraphNodeClass(n["data"]["id"], "selected", True)
    if len(selectedNodes) == 2:
        addbtn.disabled=False
    
def clearSelectedNodes():
    for nd in selectedNodes:
        toggleCytoGraphNodeClass(nd["data"]["id"], "selected", False)
    selectedNodes.clear()
    addbtn.disabled = True
    
def clearSelectedEdges():
    selectedEdges.clear()
    deletebtn.disabled = True
    changebtn.disabled = True

def addEdge(b):
    if len(selectedNodes) != 2:
        return
    n1id = selectedNodes[0]["data"]["id"]
    n2id = selectedNodes[1]["data"]["id"]
     
    n1 = getGraphNodeByName(graph, n1id)
    n2 = getGraphNodeByName(graph, n2id)
    
    exists = graph.existsDirectedPathFromTo(n1, n2) or graph.existsDirectedPathFromTo(n2, n1)
    with out:
        print(exists)
    
    if not exists:
        graph.addDirectedEdge(n1, n2)
        with out:
            print(f'adding edge between: {n1id}, {n2id}')
        buildWidget(graph, widget1)
    else:
        with out:
            print(f'edge already exists between {n1id}, {n2id}')
    clearSelectedNodes()
    
        
    
def log_clicks(node):
    with out:
        print(f'clicked: {pformat(node)}')
    if length(newEdge) == 2:
        deletebtn[disabled] = True
    else:
        newEdge.append(node["id"])

    
def removeEdge(b):
    if len(selectedEdges) == 1:
        e = selectedEdges[0]
        with out:
            print(f'removing: {pformat(e["data"])}')
        graph.removeEdge(getGraphNodeByName(graph, e["data"]["source"]), getGraphNodeByName(graph, e["data"]["target"]))
        clearSelectedEdges()
        buildWidget(graph, widget1)
        

        
def changeDirection(b):
    if len(selectedEdges) == 1:
        e = selectedEdges[0]
        source = e["data"]["source"]
        target = e["data"]["target"]
        n1 = getGraphNodeByName(graph, source)
        n2 = getGraphNodeByName(graph, target)
        graph.removeEdge(n1, n2)
        graph.addDirectedEdge(n2, n1)
        clearSelectedEdges()
        buildWidget(graph, widget1)
        ne = getCytoGraphEdge(f'{target}_{source}')
        with out:
            print(f'edge with new direction: {pformat(ne.data)}')

"""
def log_mouseovers(node):
    with out:
        print(f'mouseover: {pformat(node)}')
"""




widget1.on('edge', 'click', edge_selected)
widget1.on('node', 'click', node_selected)
#widget1.on('node', 'mouseover', log_mouseovers)
deletebtn.on_click(callback=removeEdge)
changebtn.on_click(callback=changeDirection)
addbtn.on_click(callback=addEdge)

# pc.stop_vm()

In [17]:

display(widget1)
display(deletebtn)
display(changebtn)
display(addbtn)
display(graphbtn)
display(out)

"""
Original graph:

www3.cs.stonybrook.edu/~mueller/papers/causalAnalyst.pdf
"""



CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

Button(description='Delete Edge', disabled=True, style=ButtonStyle())

Button(description='Change Direction', disabled=True, style=ButtonStyle())

Button(description='Add Edge', disabled=True, style=ButtonStyle())

Button(description='Generate original graph', style=ButtonStyle())

Output()

'\nOriginal graph:\n\nwww3.cs.stonybrook.edu/~mueller/papers/causalAnalyst.pdf\n'